In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate
from tensorflow.keras.models import Model
from keras import metrics
import keras
import os
import random
from tqdm import tqdm 
from zipfile import ZipFile
from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
import pandas as pd

In [33]:
import tensorflow as tf

class UNet(tf.keras.Model):
    def __init__(self, input_shape_, activation_="relu"):
        super(UNet, self).__init__()
        self.activation = activation_
        self.input_shape_ = input_shape_
        
        self.kernel_initializer = 'he_uniform'
        
        self.conv1a = tf.keras.layers.Conv2D(16, (3, 3), activation=self.activation, kernel_initializer=self.kernel_initializer, padding='same')
        self.conv1b = tf.keras.layers.Conv2D(16, (3, 3), activation=self.activation, kernel_initializer=self.kernel_initializer, padding='same')
        self.pool1 = tf.keras.layers.MaxPooling2D((2, 2))

        self.conv2a = tf.keras.layers.Conv2D(32, (3, 3), activation=self.activation, kernel_initializer=self.kernel_initializer, padding='same')
        self.conv2b = tf.keras.layers.Conv2D(32, (3, 3), activation=self.activation, kernel_initializer=self.kernel_initializer, padding='same')
        self.pool2 = tf.keras.layers.MaxPooling2D((2, 2))

        self.conv3a = tf.keras.layers.Conv2D(64, (3, 3), activation=self.activation, kernel_initializer=self.kernel_initializer, padding='same')
        self.conv3b = tf.keras.layers.Conv2D(64, (3, 3), activation=self.activation, kernel_initializer=self.kernel_initializer, padding='same')
        self.pool3 = tf.keras.layers.MaxPooling2D((2, 2))

        self.conv4a = tf.keras.layers.Conv2D(128, (3, 3), activation=self.activation, kernel_initializer=self.kernel_initializer, padding='same')
        self.conv4b = tf.keras.layers.Conv2D(128, (3, 3), activation=self.activation, kernel_initializer=self.kernel_initializer, padding='same')
        self.pool4 = tf.keras.layers.MaxPooling2D((2, 2))

        self.conv5a = tf.keras.layers.Conv2D(256, (3, 3), activation=self.activation, kernel_initializer=self.kernel_initializer, padding='same')
        self.conv5b = tf.keras.layers.Conv2D(256, (3, 3), activation=self.activation, kernel_initializer=self.kernel_initializer, padding='same')

        self.up6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')
        self.concat6 = tf.keras.layers.Concatenate()
        self.conv6a = tf.keras.layers.Conv2D(128, (3, 3), activation=self.activation, kernel_initializer=self.kernel_initializer, padding='same')
        self.conv6b = tf.keras.layers.Conv2D(128, (3, 3), activation=self.activation, kernel_initializer=self.kernel_initializer, padding='same')

        self.up7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')
        self.concat7 = tf.keras.layers.Concatenate()
        self.conv7a = tf.keras.layers.Conv2D(64, (3, 3), activation=self.activation, kernel_initializer=self.kernel_initializer, padding='same')
        self.conv7b = tf.keras.layers.Conv2D(64, (3, 3), activation=self.activation, kernel_initializer=self.kernel_initializer, padding='same')

        self.up8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')
        self.up8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')
        self.concat8 = tf.keras.layers.Concatenate()
        self.conv8a = tf.keras.layers.Conv2D(32, (3, 3), activation=self.activation, kernel_initializer=self.kernel_initializer, padding='same')
        self.conv8b = tf.keras.layers.Conv2D(32, (3, 3), activation=self.activation, kernel_initializer=self.kernel_initializer, padding='same')

        self.up9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')
        self.concat9 = tf.keras.layers.Concatenate()
        self.conv9a = tf.keras.layers.Conv2D(16, (3, 3), activation=self.activation, kernel_initializer=self.kernel_initializer, padding='same')
        self.conv9b = tf.keras.layers.Conv2D(16, (3, 3), activation=self.activation, kernel_initializer=self.kernel_initializer, padding='same')

        self.output_layer = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')

    def call(self, inputs):
        # Block - 1
        s = inputs

        # Contraction path
        c1 = self.conv1a(s)
        c1 = self.conv1b(c1)
        p1 = self.pool1(c1)

        c2 = self.conv2a(p1)
        c2 = self.conv2b(c2)
        p2 = self.pool2(c2)

        c3 = self.conv3a(p2)
        c3 = self.conv3b(c3)
        p3 = self.pool3(c3)

        c4 = self.conv4a(p3)
        c4 = self.conv4b(c4)
        p4 = self.pool4(c4)

        c5 = self.conv5a(p4)
        c5 = self.conv5b(c5)

        # Expansive path
        u6 = self.up6(c5)
        u6 = self.concat6([u6, c4])
        c6 = self.conv6a(u6)
        c6 = self.conv6b(c6)

        u7 = self.up7(c6)
        u7 = self.concat7([u7, c3])
        c7 = self.conv7a(u7)
        c7 = self.conv7b(c7)

        u8 = self.up8(c7)
        u8 = self.concat8([u8, c2])
        c8 = self.conv8a(u8)
        c8 = self.conv8b(c8)

        u9 = self.up9(c8)
        u9 = self.concat9([u9, c1])
        c9 = self.conv9a(u9)
        c9 = self.conv9b(c9)

        outputs = self.output_layer(c9)

        return outputs

    def dice_coef(self, y_true, y_pred, smooth=100):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)

        y_true_f = tf.keras.layers.Flatten()(y_true)
        y_pred_f = tf.keras.layers.Flatten()(y_pred)
        intersection = tf.reduce_sum(y_true_f * y_pred_f)
        dice = (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)
        return dice
    
    def buildModel(self):
        inputs = tf.keras.Input(shape=self.input_shape_)
        outputs = self.call(inputs)
        model = tf.keras.Model(inputs=inputs, outputs=outputs)
        return model

    def dice_coef(self, y_true, y_pred, smooth=100):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)

        y_true_f = tf.keras.layers.Flatten()(y_true)
        y_pred_f = tf.keras.layers.Flatten()(y_pred)
        intersection = tf.reduce_sum(y_true_f * y_pred_f)
        dice = (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)
        return dice

    def dice_coef_loss(self, y_true, y_pred):
        return -self.dice_coef(y_true, y_pred)

    def iou(self, y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)

        intersection = tf.reduce_sum(tf.multiply(y_true, y_pred))
        union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
        iou = (intersection + 1e-15) / (union + 1e-15)
        return iou

    def CompileandSummarize(self, model_):
        model_.compile(optimizer=tf.keras.optimizers.Adam(), loss=self.dice_coef_loss, metrics=[self.iou])
        model_.summary()
        return model_




In [13]:
'''Defining callbacks to calculate training time '''

from timeit import default_timer as timer

class TimingCallback(keras.callbacks.Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)


In [14]:
# Now lets focus on preparing the data

with ZipFile("../input/data-science-bowl-2018/stage1_train.zip","r") as zip_ref:
    zip_ref.extractall("./stage1_train")
    
with ZipFile("../input/data-science-bowl-2018/stage1_test.zip","r") as zip_ref:
    zip_ref.extractall("./stage1_test")    

In [15]:
seed = 42
np.random.seed = seed

IMG_WIDTH = 96
IMG_HEIGHT = 96
IMG_CHANNELS = 3

TRAIN_PATH = './stage1_train/'
TEST_PATH = './stage1_test/'

In [16]:
train_ids = next(os.walk(TRAIN_PATH))[1]
test_ids = next(os.walk(TEST_PATH))[1]


In [17]:
X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)

print('Resizing training images and masks')
for n, file_name in tqdm(enumerate(train_ids)):   
    path = TRAIN_PATH + file_name
    img = imread(path + '/images/' + file_name + '.png')[:,:,:IMG_CHANNELS]  
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_train[n] = img  #Fill empty X_train with values from img
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    for mask_file in next(os.walk(path + '/masks/'))[2]:
        mask_ = imread(path + '/masks/' + mask_file)
        mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant',  
                                      preserve_range=True), axis=-1)
        mask = np.maximum(mask, mask_)  
            
    Y_train[n] = mask

/tmp/ipykernel_31/2570245034.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)


Resizing training images and masks


0it [00:00, ?it/s]/tmp/ipykernel_31/2570245034.py:10: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
670it [04:52,  2.29it/s]


In [20]:
# test images
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
sizes_test = []
print('Resizing test images') 
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = TEST_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img


Resizing test images


100%|██████████| 65/65 [00:01<00:00, 45.42it/s]


In [21]:
class GCU(tf.keras.layers.Layer):
    def __init__(self):
        super(GCU, self).__init__()
    
    def call(self, x):
        return tf.cast(tf.math.multiply(x , tf.math.cos(x))/tf.reduce_sum(x), tf.float32)

In [34]:

training_inferences = {}
cb = TimingCallback()

UNet_1 = UNet((96, 96, 3), activation_= "relu")
model1 = UNet_1.buildModel()
UNet_1.CompileandSummarize(model1)
results1 = model1.fit(x = X_train, y = Y_train, batch_size = 8, epochs=100, callbacks = [cb])

training_inferences['ReLU'] = sum(cb.logs)


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 96, 96, 3)]  0           []                               
                                                                                                  
 conv2d_133 (Conv2D)            (None, 96, 96, 16)   448         ['input_2[0][0]']                
                                                                                                  
 conv2d_134 (Conv2D)            (None, 96, 96, 16)   2320        ['conv2d_133[0][0]']             
                                                                                                  
 max_pooling2d_28 (MaxPooling2D  (None, 48, 48, 16)  0           ['conv2d_134[0][0]']             
 )                                                                                          